In [1]:
!pip install ultralytics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.5/485.5 KB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.1/189.1 KB 23.6 MB/s eta 0:00:00


In [2]:
import cv2
import os
import shutil
from ultralytics import YOLO
import cv2
from google.colab.patches import cv2_imshow

In [ ]:
from google.colab import drive

drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [15]:
RootDir="/content/gdrive/My Drive/ColabNotebooks/MenOnCamera/"
model_path = f"{RootDir}runs/detect/train_150ep_1640ph/weights/last.pt"

img_path = f"{RootDir}data/images/test/RGtest3"
label_path = f"{RootDir}data/labels/test/RGtest3"
files=os.listdir(img_path)
if "result" not in files:
  os.makedirs(os.path.join(img_path,"result"))

model = YOLO(model_path) 

In [ ]:
def ManDetection(threshold):
  files=os.listdir(img_path)
  print(files)
  for img in files:
    imgData=[]

    if img[-4:]==".jpg":
      img2 = cv2.imread(os.path.join(img_path,img))
      H, W, _ = img2.shape
      print(H,W)
      img2=cv2.cvtColor(img2,cv2.COLOR_BGR2RGB)
      results  = model (img2) [0]

      for result in results.boxes.data.tolist():
        x1, y1, x2, y2, score, class_id = result
        

        if score>=threshold:
          cv2.rectangle (img2, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 3)
          cv2.putText(img2, "man", (int(x1), int(y1 - 10)),
                cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 255, 0), 2, cv2.LINE_AA)
          
          print(int(x1), int(y1), int(x2), int(y2), score, class_id)
          x1,x2,y1,y2=x1/W,x2/W,y1/H,y2/H
          dx=(x2-x1)/2
          dy=(y2-y1)/2
          x3=x1+dx
          y3=y1+dy
          imgData.append(f"0 {x3} {y3} {dx} {dy}")

      cv2_imshow(img2)
      if len(imgData)!=0:
        cv2.imwrite(os.path.join(img_path,"result",img),img2)
        path=os.path.join(label_path,img[:-4]+".txt")
        with open(path,"w") as file_txt:
          for data in imgData:
            file_txt.write(data+"\n")

In [ ]:
ManDetection(0.5)

Output hidden; open in https://colab.research.google.com to view.